# Importación de módulos

In [1]:
import os
import pandas as pd

# Lectura de la Base de Datos
Para descargar la base de datos original se necesita ir a la siguiente [ruta](https://sio.cnsf.gob.mx/)

## Definición de ruta

In [2]:
read_dir = r'C:\Users\ed_22\Documents\SIO\Bases Originales'
read_file = 'ER_dic2023.xlsx'
read_path = os.path.join(read_dir, read_file)

## Variables globales

In [3]:
FIELDS = {
    'FECHA_CORTE':'corte',
    'NOMBRE_CORTO':'nom_inst',
    'DESCRIPCION':'desc',
    'DESC_OPERACION':'desc_op',
    'IMPORTE':'importe'
}

In [4]:
ER = pd.read_excel(read_path, usecols=FIELDS.keys()).rename(columns=FIELDS)

# Análisis exploratorio

In [5]:
ER.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536897 entries, 0 to 536896
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   corte     536897 non-null  datetime64[ns]
 1   nom_inst  536897 non-null  object        
 2   desc      536897 non-null  object        
 3   desc_op   536897 non-null  object        
 4   importe   536897 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 20.5+ MB


In [6]:
ER.nom_inst.unique()

array(['Crédito Afianzador', 'Fianzas Guardiana Inbursa',
       'Liberty Fianzas', 'Fianzas Asecam', 'Afianzadora CBL Fiducia',
       'Mapfre Fianzas', 'CESCE Fianzas México', 'Zurich Fianzas México',
       'MBIA México', 'Plan Seguro', 'Medi Access Seguros de Salud',
       'BBVA Bancomer Seguros Salud', 'AXA Salud', 'General de Salud',
       'Servicios Integrales de Salud Nova', 'Seguros Centauro',
       'Dentegra Seguros Dentales', 'Odontored Seguros Dentales',
       'Pensiones Sura', 'Pensiones Banorte', 'HSBC Pensiones',
       'Pensiones BBVA Bancomer', 'Profuturo GNP Pensiones',
       'Principal Pensiones', 'Pensiones Banamex',
       'Metlife Pensiones México', 'Pensiones Inbursa', 'Seguros Banorte',
       'Allianz México', 'Patrimonial Inbursa',
       'Atradius Seguros de Crédito', 'Seguros El Potosí',
       'General de Seguros', 'Seguros Sura', 'AIG Seguros México',
       'La Latinoamericana Seguros', 'Seguros Ve por Más',
       'Zurich Santander Seguros México', 

In [7]:
ER.desc.unique()

array(['Directo', 'Primas Cedidas', 'Primas De Retención',
       'Primas de Retención Devengadas', 'Costo Neto de Adquisición',
       'Comisiones a Agentes',
       'Costo Neto de Siniestralidad, Reclamaciones y Otras Obligaciones Pendientes de Cumplir',
       'Utilidad (Pérdida) Técnica', 'Gastos de Operación Netos',
       'Resultado Integral de Financiamiento',
       'Utilidad (Pérdida) antes de Impuestos a la Utilidad',
       'Utilidad  (Pérdida) del Ejercicio',
       'Utilidad (Pérdida) de la Operación', 'Tomado',
       'Compensaciones Adicionales a Agentes'], dtype=object)

**Nota**: Hay doble espacio en el concepto `'Utilidad  (Pérdida) del Ejercicio'`

In [9]:
ER.desc_op.unique()

array(['Total', '*Fianzas', '**Fidelidad', '***Individual',
       '***Colectivo', '**Judicial o Penal', '***Penales',
       '***No penales', '***Amparan conductores de automóviles',
       '**Administrativo', '***Por Obra', '***Proveeduría', '***Fiscales',
       '***Arrendamiento', '***Otras de Administrativo', '**De crédito',
       '***Suministro', '***Compra Venta', '***Otras de Crédito',
       '***Financiera', '*Da?os', '**Garantía financiera',
       '*Accidentes y Enfermedades', '***Gastos Médicos', '***Salud',
       '***Accidentes Personales', '*Pensiones',
       '***IMSS Incapacidad Permanente RT',
       '***IMSS Muerte (riesgos de trabajo)',
       '***IMSS Invalidez (invalidez y vida)',
       '***IMSS Muerte (invalidez y vida)',
       '***IMSS Jubilación (retiro, cesantía y vejez)',
       '***ISSSTE Incapacidad Permanente RT',
       '***ISSSTE Muerte (riesgos de trabajo)',
       '***ISSSTE Invalidez (invalidez y vida)',
       '***ISSSTE Muerte (invalidez y vida)'

**Notas**
- Los caracteres `'ñ'` se ponen como `'?'`
- Símbolos `'*'`

# Limpieza
¡Buena práctica!

Haz una copia de tu base de datos original.

In [10]:
df = ER.copy()

In [11]:
df['nom_inst'] = df.nom_inst.str.replace('?', 'ñ', regex=True)
df['desc'] = df.desc.str.replace('?', 'ñ', regex=True)
df['desc_op'] = df.desc_op.str.replace('*', '', regex=True)
df['desc_op'] = df.desc_op.str.replace('?', 'ñ', regex=True)

# Reduciendo tamaño
## Creando catálogos
¡Buenas prácticas!

- Usa el **tipado** de Python (ver [referencia](https://docs.python.org/3/library/typing.html))
- Docstrings

In [12]:
def create_db(field: pd.Series, pk_name: str, field_name: str) -> pd.DataFrame:
    """
    Create a DataFrame representing a database with unique identifiers based on a given field.

    Parameters:
    field (pandas.Series): The field from which unique identifiers will be generated.
    pk (str): The name of the column for unique identifiers.
    field_name (str): The name of the column for the original field values.

    Returns:
    pandas.DataFrame: A DataFrame representing the database with unique identifiers.

    Example:
    >>> import pandas as pd
    >>> data = {'field': ['a', 'b', 'c', 'a', 'b']}
    >>> df = pd.DataFrame(data)
    >>> create_db(df['field'], 'ID', 'Field_Name')
       ID Field_Name
    0   1          a
    1   2          b
    2   3          c
    """
    # Generate unique identifiers for each unique value in the field
    unique_values = field.unique()
    identifiers = range(1, len(unique_values) + 1)

    # Create a DataFrame with the primary key and field name columns
    db_df = pd.DataFrame({pk_name: identifiers, field_name: unique_values})
    
    return db_df

Testeando

In [13]:
create_db(df.desc, 'id_desc', 'descripcion').head()

,id_desc,descripcion
0,1,Directo
1,2,Primas Cedidas
2,3,Primas De Retención
3,4,Primas de Retención Devengadas
4,5,Costo Neto de Adquisición


Ventajas del *docstring*

In [14]:
help(create_db)

Help on function create_db in module __main__:

create_db(field: pandas.core.series.Series, pk_name: str, field_name: str) -> pandas.core.frame.DataFrame
    Create a DataFrame representing a database with unique identifiers based on a given field.
    
    Parameters:
    field (pandas.Series): The field from which unique identifiers will be generated.
    pk (str): The name of the column for unique identifiers.
    field_name (str): The name of the column for the original field values.
    
    Returns:
    pandas.DataFrame: A DataFrame representing the database with unique identifiers.
    
    Example:
    >>> import pandas as pd
    >>> data = {'field': ['a', 'b', 'c', 'a', 'b']}
    >>> df = pd.DataFrame(data)
    >>> create_db(df['field'], 'ID', 'Field_Name')
       ID Field_Name
    0   1          a
    1   2          b
    2   3          c



In [15]:
print(create_db.__doc__)


    Create a DataFrame representing a database with unique identifiers based on a given field.

    Parameters:
    field (pandas.Series): The field from which unique identifiers will be generated.
    pk (str): The name of the column for unique identifiers.
    field_name (str): The name of the column for the original field values.

    Returns:
    pandas.DataFrame: A DataFrame representing the database with unique identifiers.

    Example:
    >>> import pandas as pd
    >>> data = {'field': ['a', 'b', 'c', 'a', 'b']}
    >>> df = pd.DataFrame(data)
    >>> create_db(df['field'], 'ID', 'Field_Name')
       ID Field_Name
    0   1          a
    1   2          b
    2   3          c
    


## Creamos tablas de catálogos

In [16]:
# nombres de instituciones
instituciones = create_db(df.nom_inst, pk_name = 'id_nom', field_name = 'nombre')
# trimestres
trimestres = create_db(df.corte, pk_name = 'id_trim', field_name = 'trimestre')
# descripciones de cuentas
cuentas = create_db(df.desc, pk_name = 'id_cuenta', field_name = 'cuenta')
# descripciones de operaciones
operaciones = create_db(df.desc_op, pk_name = 'id_op', field_name = 'operacion')

In [17]:
print('Instituciones', instituciones.head(), sep = '\n')
print(60*'-')
print('Trimestres', trimestres.head(), sep = '\n')
print(60*'-')
print('Cuentas', cuentas.head(), sep = '\n')
print(60*'-')
print('Operaciones', operaciones.head(), sep = '\n')

Instituciones
   id_nom                     nombre
0       1         Crédito Afianzador
1       2  Fianzas Guardiana Inbursa
2       3            Liberty Fianzas
3       4             Fianzas Asecam
4       5    Afianzadora CBL Fiducia
------------------------------------------------------------
Trimestres
   id_trim  trimestre
0        1 2016-03-31
1        2 2016-06-30
2        3 2016-09-30
3        4 2016-12-31
4        5 2017-03-31
------------------------------------------------------------
Cuentas
   id_cuenta                          cuenta
0          1                         Directo
1          2                  Primas Cedidas
2          3             Primas De Retención
3          4  Primas de Retención Devengadas
4          5       Costo Neto de Adquisición
------------------------------------------------------------
Operaciones
   id_op   operacion
0      1       Total
1      2     Fianzas
2      3   Fidelidad
3      4  Individual
4      5   Colectivo


## Incorporación de los identificadores
Los famosos join's

In [18]:
df.merge(instituciones, left_on = 'nom_inst', right_on = 'nombre', how = 'inner')

,corte,nom_inst,desc,desc_op,importe,id_nom,nombre
0,2016-03-31,Crédito Afianzador,Directo,Total,17.68,1,Crédito Afianzador
1,2016-06-30,Crédito Afianzador,Directo,Total,19.30,1,Crédito Afianzador
2,2016-09-30,Crédito Afianzador,Directo,Total,21.39,1,Crédito Afianzador
3,2016-12-31,Crédito Afianzador,Directo,Total,24.54,1,Crédito Afianzador
4,2017-03-31,Crédito Afianzador,Directo,Total,17.77,1,Crédito Afianzador
...,...,...,...,...,...,...,...
536892,2022-12-31,Der Neue Horizont Re,Utilidad (Pérdida) de la Operación,Agrícola,-30.30,114,Der Neue Horizont Re
536893,2023-03-31,Der Neue Horizont Re,Utilidad (Pérdida) de la Operación,Agrícola,-6.13,114,Der Neue Horizont Re
536894,2023-06-30,Der Neue Horizont Re,Utilidad (Pérdida) de la Operación,Agrícola,-20.35,114,Der Neue Horizont Re
536895,2023-09-30,Der Neue Horizont Re,Utilidad (Pérdida) de la Operación,Agrícola,-28.50,114,Der Neue Horizont Re


In [19]:
df.\
    merge(instituciones, left_on = 'nom_inst', right_on = 'nombre', how = 'inner').\
    merge(trimestres, left_on = 'corte', right_on = 'trimestre', how = 'inner').\
    merge(cuentas, left_on = 'desc', right_on = 'cuenta', how = 'inner').\
    merge(operaciones, left_on = 'desc_op', right_on = 'operacion', how = 'inner')

,corte,nom_inst,desc,desc_op,importe,id_nom,nombre,id_trim,trimestre,id_cuenta,cuenta,id_op,operacion
0,2016-03-31,Crédito Afianzador,Directo,Total,17.68,1,Crédito Afianzador,1,2016-03-31,1,Directo,1,Total
1,2016-03-31,Fianzas Guardiana Inbursa,Directo,Total,496.03,2,Fianzas Guardiana Inbursa,1,2016-03-31,1,Directo,1,Total
2,2016-03-31,Liberty Fianzas,Directo,Total,111.25,3,Liberty Fianzas,1,2016-03-31,1,Directo,1,Total
3,2016-03-31,Fianzas Asecam,Directo,Total,10.07,4,Fianzas Asecam,1,2016-03-31,1,Directo,1,Total
4,2016-03-31,Afianzadora CBL Fiducia,Directo,Total,11.29,5,Afianzadora CBL Fiducia,1,2016-03-31,1,Directo,1,Total
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536892,2022-03-31,Chubb Fianzas Monterrey,Utilidad (Pérdida) de la Operación,De Crédito,-0.03,107,Chubb Fianzas Monterrey,25,2022-03-31,13,Utilidad (Pérdida) de la Operación,69,De Crédito
536893,2022-06-30,Chubb Fianzas Monterrey,Utilidad (Pérdida) de la Operación,De Crédito,-0.04,107,Chubb Fianzas Monterrey,26,2022-06-30,13,Utilidad (Pérdida) de la Operación,69,De Crédito
536894,2022-09-30,Chubb Fianzas Monterrey,Utilidad (Pérdida) de la Operación,De Crédito,-0.07,107,Chubb Fianzas Monterrey,27,2022-09-30,13,Utilidad (Pérdida) de la Operación,69,De Crédito
536895,2022-12-31,Chubb Fianzas Monterrey,Utilidad (Pérdida) de la Operación,De Crédito,-0.08,107,Chubb Fianzas Monterrey,28,2022-12-31,13,Utilidad (Pérdida) de la Operación,69,De Crédito


In [20]:
df = df.\
        merge(instituciones, left_on = 'nom_inst', right_on = 'nombre', how = 'inner').\
        merge(trimestres, left_on = 'corte', right_on = 'trimestre', how = 'inner').\
        merge(cuentas, left_on = 'desc', right_on = 'cuenta', how = 'inner').\
        merge(operaciones, left_on = 'desc_op', right_on = 'operacion', how = 'inner')
df = df[['id_trim', 'id_nom', 'id_cuenta', 'id_op', 'importe']]
df.head()

,id_trim,id_nom,id_cuenta,id_op,importe
0,1,1,1,1,17.68
1,1,2,1,1,496.03
2,1,3,1,1,111.25
3,1,4,1,1,10.07
4,1,5,1,1,11.29


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536897 entries, 0 to 536896
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id_trim    536897 non-null  int64  
 1   id_nom     536897 non-null  int64  
 2   id_cuenta  536897 non-null  int64  
 3   id_op      536897 non-null  int64  
 4   importe    536897 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 24.6 MB


¿Por qué esto nos reduce tamaño?

¿Nos reduce tamaño?

In [22]:
# base completa
save_path = os.path.join(read_dir, 'dic2023.xlsx')
df.to_excel(save_path, index = False)

In [23]:
# trimestre
save_path = os.path.join(read_dir, 'trimestres.xlsx')
trimestres.columns = ['id', 'nombre']
trimestres.to_excel(save_path, index = False)
# instituciones
save_path = os.path.join(read_dir, 'instituciones.xlsx')
instituciones.columns = ['id', 'nombre']
instituciones.to_excel(save_path, index = False)
# cuentas
save_path = os.path.join(read_dir, 'cuentas.xlsx')
cuentas.columns = ['id', 'nombre']
cuentas.to_excel(save_path, index = False)
# operaciones
save_path = os.path.join(read_dir, 'operaciones.xlsx')
operaciones.columns = ['id', 'nombre']
operaciones.to_excel(save_path, index = False)